In [2]:
import numpy as np


class Particle:
    

    def __init__(self, no_dim, x_range, v_range):
        
        self.x = np.random.uniform(
            x_range[0], x_range[1], (no_dim,)
        )  # particle position in each dimension...
        self.v = np.random.uniform(
            v_range[0], v_range[1], (no_dim,)
        )  # particle velocity in each dimension...
        self.pbest = np.inf
        self.pbestpos = np.zeros((no_dim,))


class Swarm:

    def __init__(self, no_particle, no_dim, x_range, v_range, iw_range, c):
        
        self.p = np.array(
            [Particle(no_dim, x_range, v_range) for i in range(no_particle)]
        )
        self.gbest = np.inf
        self.gbestpos = np.zeros((no_dim,))
        self.x_range = x_range
        self.v_range = v_range
        self.iw_range = iw_range
        self.c0 = c[0]
        self.c1 = c[1]
        self.no_dim = no_dim

    def optimize(self, function, X, Y, print_step, iter):
        
        for i in range(iter):
            for particle in self.p:
                fitness = function(X, Y, particle.x)

                if fitness < particle.pbest:
                    particle.pbest = fitness
                    particle.pbestpos = particle.x.copy()

                if fitness < self.gbest:
                    self.gbest = fitness
                    self.gbestpos = particle.x.copy()

            for particle in self.p:
                # Here iw is inertia weight...
                iw = np.random.uniform(self.iw_range[0], self.iw_range[1], 1)[0]
                particle.v = (
                    iw * particle.v
                    + (
                        self.c0
                        * np.random.uniform(0.0, 1.0, (self.no_dim,))
                        * (particle.pbestpos - particle.x)
                    )
                    + (
                        self.c1
                        * np.random.uniform(0.0, 1.0, (self.no_dim,))
                        * (self.gbestpos - particle.x)
                    )
                )
                # particle.v = particle.v.clip(min=self.v_range[0], max=self.v_range[1])
                particle.x = particle.x + particle.v
                # particle.x = particle.x.clip(min=self.x_range[0], max=self.x_range[1])

            if i % print_step == 0:
                print("iteration#: ", i + 1, " loss: ", fitness)

        print("global best loss: ", self.gbest)

    def get_best_solution(self):
       
        return self.gbestpos

In [15]:
INPUT_NODES = 4
HIDDEN_NODES = 20
OUTPUT_NODES = 3


def one_hot_encode(Y):
    
    num_unique = len(np.unique(np.array(Y)))
    zeros = np.zeros((len(Y), num_unique))
    zeros[range(len(Y)), Y] = 1
    return zeros


def softmax(logits):
    
    exps = np.exp(logits)
    return exps / np.sum(exps, axis=1, keepdims=True)


def Negative_Likelihood(probs, Y):
   
    num_samples = len(probs)
    corect_logprobs = -np.log(probs[range(num_samples), Y])
    return np.sum(corect_logprobs) / num_samples


def Cross_Entropy(probs, Y):
   
    num_samples = len(probs)
    ind_loss = np.max(-1 * Y * np.log(probs + 1e-12), axis=1)
    return np.sum(ind_loss) / num_samples


def forward_pass(X, Y, W):

    if isinstance(W, Particle):
        W = W.x

    w1 = W[0 : INPUT_NODES * HIDDEN_NODES].reshape((INPUT_NODES, HIDDEN_NODES))
    b1 = W[
        INPUT_NODES * HIDDEN_NODES : (INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES
    ].reshape((HIDDEN_NODES,))
    w2 = W[
        (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES : (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES
        + (HIDDEN_NODES * OUTPUT_NODES)
    ].reshape((HIDDEN_NODES, OUTPUT_NODES))
    b2 = W[
        (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES
        + (HIDDEN_NODES * OUTPUT_NODES) : (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES
        + (HIDDEN_NODES * OUTPUT_NODES)
        + OUTPUT_NODES
    ].reshape((OUTPUT_NODES,))

    z1 = np.dot(X, w1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, w2) + b2
    logits = z2

    probs = softmax(logits)

    return Negative_Likelihood(probs, Y)
   


def predict(X, W):
   

    w1 = W[0 : INPUT_NODES * HIDDEN_NODES].reshape((INPUT_NODES, HIDDEN_NODES))
    b1 = W[
        INPUT_NODES * HIDDEN_NODES : (INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES
    ].reshape((HIDDEN_NODES,))
    w2 = W[
        (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES : (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES
        + (HIDDEN_NODES * OUTPUT_NODES)
    ].reshape((HIDDEN_NODES, OUTPUT_NODES))
    b2 = W[
        (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES
        + (HIDDEN_NODES * OUTPUT_NODES) : (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES
        + (HIDDEN_NODES * OUTPUT_NODES)
        + OUTPUT_NODES
    ].reshape((OUTPUT_NODES,))

    z1 = np.dot(X, w1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, w2) + b2
    logits = z2

    probs = softmax(logits)
    Y_pred = np.argmax(probs, axis=1)
    return Y_pred


def get_accuracy(Y, Y_pred):
    
    return (Y == Y_pred).mean()
    
from sklearn.datasets import load_iris
data = load_iris()


X = data.data
Y = data.target

if __name__ == "__main__":
    no_solution = 100
    no_dim = (
        (INPUT_NODES * HIDDEN_NODES)
        + HIDDEN_NODES
        + (HIDDEN_NODES * OUTPUT_NODES)
        + OUTPUT_NODES
    )
    w_range = (0.0, 1.0)
    lr_range = (0.0, 1.0)
    iw_range = (0.9, 0.9)  
    c = (0.5, 0.3)  

    s = Swarm(no_solution, no_dim, w_range, lr_range, iw_range, c)
    
    s.optimize(forward_pass, X, Y, 100, 1000)
    W = s.get_best_solution()
    Y_pred = predict(X, W)
    accuracy = get_accuracy(Y, Y_pred)
    print("Accuracy: %.3f" % accuracy)

iteration#:  1  loss:  1.4257736814734927
iteration#:  101  loss:  0.05797500129847069
iteration#:  201  loss:  0.03634923885286586
iteration#:  301  loss:  0.034720627693649316
iteration#:  401  loss:  0.03098635562739237
iteration#:  501  loss:  0.029913847392940045
iteration#:  601  loss:  0.029109788341715112
iteration#:  701  loss:  0.05991443286993808
iteration#:  801  loss:  0.033841247823821474
iteration#:  901  loss:  0.03767666432784546
global best loss:  0.028185045001252275
Accuracy: 0.993


In [6]:
import random
import logging
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(message)s',
                    handlers=[logging.FileHandler("ann_test.log"),
                              logging.StreamHandler()])

class ANN(Sequential):
    
    def __init__(self, child_weights=None):
        super().__init__()

        if child_weights is None:
            layer1 = Dense(8, input_shape=(8,), activation='sigmoid')
            layer2 = Dense(1, activation='sigmoid')
            self.add(layer1)
            self.add(layer2)
        else:
            self.add(
                Dense(
                    8,
                    input_shape=(8,),
                    activation='sigmoid',
                    weights=[child_weights[0], np.ones(8)])
                )
            self.add(
                Dense(
                    1,
                    activation='sigmoid',
                    weights=[child_weights[1], np.zeros(1)])
            )

    def forward_propagation(self, train_feature, train_label):
        predict_label = self.predict(train_feature.values)
        self.fitness = accuracy_score(train_label, predict_label.round())

def crossover(nn1, nn2):
    
    nn1_weights = []
    nn2_weights = []
    child_weights = []

    for layer in nn1.layers:
        nn1_weights.append(layer.get_weights()[0])

    for layer in nn2.layers:
        nn2_weights.append(layer.get_weights()[0])

    for i in range(len(nn1_weights)):
        # Get single point to split the matrix in parents based on # of cols
        split = random.randint(0, np.shape(nn1_weights[i])[1]-1)
        # Iterate through after a single point and set the remaing cols to nn_2
        for j in range(split, np.shape(nn1_weights[i])[1]-1):
            nn1_weights[i][:, j] = nn2_weights[i][:, j]

        child_weights.append(nn1_weights[i])

    mutation(child_weights)

    child = ANN(child_weights)
    return child

def mutation(child_weights):
    selection = random.randint(0, len(child_weights)-1)
    mut = random.uniform(0, 1)
    if mut <= .05:
        child_weights[selection] *= random.randint(2, 5)
    else:
        pass

# Preprocess Data
df = pd.read_table('./diabetes.txt',header=None,encoding='gb2312',sep='\t')
df.astype(float)
# remove redundant col which is the opposite value of the 10th col
df.pop(10)
# remove first col of bias = 1
df.pop(0)
# the label column
label = df.pop(9)
# train feature
train_feature = df[:576]
# train label
train_label = label[:576]
# test feature
test_feature = df[576:]
# test label
test_label = label[576:]

# store all active ANNs
networks = []
pool = []
# Generation counter
generation = 0
# Initial Population
population = 10
for i in range(population):
    networks.append(ANN())
# Track Max Fitness
max_fitness = 0
# Store Max Fitness Weights
optimal_weights = []

epochs = 10
# Evolution Loop
for i in range(epochs):
    generation += 1
    logging.debug("Generation: " + str(generation) + "\r\n")

    for ann in networks:
        # Propagate to calculate fitness score
        ann.forward_propagation(train_feature, train_label)
        # Add to pool after calculating fitness
        pool.append(ann)

    # Clear for propagation of next children
    networks.clear()

    # Sort anns by fitness
    pool = sorted(pool, key=lambda x: x.fitness)
    pool.reverse()

    # Find Max Fitness and Log Associated Weights
    for i in range(len(pool)):
        if pool[i].fitness > max_fitness:
            max_fitness = pool[i].fitness

            logging.debug("Max Fitness: " + str(max_fitness) + "\r\n")

            # Iterate through layers, get weights, and append to optimal
            optimal_weights = []
            for layer in pool[i].layers:
                optimal_weights.append(layer.get_weights()[0])
            logging.debug('optimal_weights: ' + str(optimal_weights)+"\r\n")

    # Crossover: top 5 randomly select 2 partners
    for i in range(5):
        for j in range(2):
            # Create a child and add to networks
            temp = crossover(pool[i], random.choice(pool))
            # Add to networks to calculate fitness score next iteration
            networks.append(temp)

# Create a Genetic Neural Network with optimal initial weights
ann = ANN(optimal_weights)
predict_label = ann.predict(test_feature.values)
print('Test Accuracy: %.2f' % accuracy_score(test_label, predict_label.round()))

2023-04-04 23:21:17,802 Generation: 1

2023-04-04 23:21:18,579 Max Fitness: 0.65625

2023-04-04 23:21:18,583 optimal_weights: [array([[-0.5563627 , -0.31606245, -0.34847614,  0.14145988,  0.5487862 ,
         0.5720741 ,  0.17325789,  0.0786643 ],
       [-0.2696759 , -0.16208503,  0.25399452,  0.3961001 ,  0.02770311,
         0.2918352 ,  0.39345688,  0.47803885],
       [ 0.4316457 , -0.47055513, -0.32839575,  0.34939903, -0.5443043 ,
         0.10570222,  0.02974683,  0.41953915],
       [ 0.33310091, -0.46281737, -0.5099954 ,  0.40223116,  0.06806213,
        -0.42415443,  0.28390616,  0.5393463 ],
       [-0.6033255 , -0.04578722, -0.28090075,  0.22651279, -0.03428686,
         0.24592036,  0.34704638, -0.19846979],
       [ 0.4094829 , -0.1970619 , -0.4465279 , -0.24428198,  0.51600116,
         0.32268858, -0.41697383, -0.12745884],
       [-0.5655789 , -0.23089644,  0.30102468,  0.00151068,  0.24946618,
         0.10389328, -0.39318398, -0.13610005],
       [ 0.31638628, -0.54

Test Accuracy: 0.66


In [8]:
class CulAl:
    def __init__(self, num_inputs, num_hidden, num_outputs, num_knowledges, knowledge_threshold):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.num_knowledges = num_knowledges
        self.knowledge_threshold = knowledge_threshold
        self.hidden_weights = np.random.randn(self.num_inputs, self.num_hidden)
        self.output_weights = np.random.randn(self.num_hidden, self.num_outputs)
        self.knowledges = []

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def predict(self, inputs):
        hidden_layer = self.sigmoid(np.dot(inputs, self.hidden_weights))
        output_layer = self.sigmoid(np.dot(hidden_layer, self.output_weights))
        return output_layer

    def calculate_fitness(self, X, y):
        predictions = self.predict(X)
        fitness = np.sum(np.abs(predictions - y))
        return fitness

    def update_knowledge(self, X, y):
        fitness = self.calculate_fitness(X, y)
        if len(self.knowledges) < self.num_knowledges or fitness < max([k["fitness"] for k in self.knowledges]):
            self.knowledges.append({"hidden_weights": self.hidden_weights.copy(), "output_weights": self.output_weights.copy(), "fitness": fitness})
        self.knowledges = sorted(self.knowledges, key=lambda k: k["fitness"])[:self.num_knowledges]

    def inherit_knowledge(self):
        if len(self.knowledges) > 0 and self.calculate_fitness(X, y) > self.knowledge_threshold:
            best_knowledge = self.knowledges[0]
            self.hidden_weights = best_knowledge["hidden_weights"].copy()
            self.output_weights = best_knowledge["output_weights"].copy()

    def train(self, X, y, population_size=10, generations=100, mutation_rate=0.1):
        population = [CulAl(self.num_inputs, self.num_hidden, self.num_outputs, self.num_knowledges, self.knowledge_threshold) for _ in range(population_size)]

        for generation in range(generations):
            for individual in population:
                individual.update_knowledge(X, y)

            for individual in population:
                individual.inherit_knowledge()

            fitnesses = [individual.calculate_fitness(X, y) for individual in population]
            fittest_idx = np.argmin(fitnesses)
            fittest = population[fittest_idx]

            if fitnesses[fittest_idx] == 0:
                print("Solution found in generation", generation)
                break

            parents = np.random.choice(population, size=population_size-1, replace=True, p=np.array(fitnesses) / np.sum(fitnesses))
            children = [fittest.crossover(parent) for parent in parents]
            population = [fittest] + children
            for individual in population:
                individual.mutate(mutation_rate)

        self.hidden_weights = fittest.hidden_weights
        self.output_weights = fittest.output_weights

    def crossover(self, other):
        child = CulAl(self.num_inputs, self.num_hidden, self.num_outputs, self.num_knowledges, self.knowledge_threshold)
        child.hidden_weights = self.hidden_weights.copy()
        child.output_weights = other.output_weights.copy()
        return child

    def mutate(self, mutation_rate):
        for i in range(self.num_inputs):
            for j in range(self.num_hidden):
                if np.random.random() < mutation_rate:
                    self.hidden_weights[i,j] += np.random.randn() * 0.1

        for i in range(self.num_hidden):
            for j in range(self.num_outputs):
                if np.random.random() < mutation_rate:
                    self.output_weights[i,j] += np.random.randn() * 0.1


In [10]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            print (shortest_path)
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone = self.pheromone * self.decay            
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # going back to where we started    
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move


In [16]:
import numpy as np



distances = np.array([[2, 2, 5, 7],
                      [4, 8, 2, 8],
                      [2, 4, 1, 3],
                      [5, 8, 1, 6]])

ant_colony = AntColony(distances, 1, 1, 100, 0.95, alpha=1, beta=1)
shortest_path = ant_colony.run()
print ("shorted_path: {}".format(shortest_path))

([(0, 3), (3, 2), (2, 1), (1, 0)], 16)
([(0, 3), (3, 1), (1, 2), (2, 0)], 19)
([(0, 3), (3, 1), (1, 2), (2, 0)], 19)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3), (3, 0)], 12)
([(0, 1), (1, 2), (2, 3),